<a href="https://colab.research.google.com/github/ronyates47/Gedcom-Utils/blob/main/another_start-2023_0917_1544_hrs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Base script-stable-selects GEDCOM-correct output
# Section 1
import glob

# Initialize the GEDCOM parser
gedcom_parser = Parser()

# Parse the GEDCOM file
gedcom_parser.parse_file(file_path)

# Get the root element of the GEDCOM file
root_elements = gedcom_parser.get_element_list()

# Initialize the Gedcom object and parse the GEDCOM file
gedcom = Gedcom(file_path)
gedcom.parse_gedcom()

# Initialize the list of individuals
individuals = []

# Iterate over the filter_pool list and add each individual's last name and ID to the individuals list
for dataset in gedcom.filter_pool:
    individual_id = dataset.get_gen_person()
    last_name = dataset.get_anchor_gen1()
    individuals.append((last_name, individual_id))

print(f'Total records found: {len(individuals)}')

# Read the GEDCOM file and split it into individual and family records
with open(file_path, 'r') as file:
    data = file.read()
data = data.split('\n0 ')
records = {extract_id(record): record for record in data}

# Global variables
generation_table = None
visited_pairs = None


class Gedcom:
    def __init__(self, file_name):
        self.file_name = file_name
        self.gedcom_datasets = []
        self.filter_pool = []

    @staticmethod
    def get_standard_name(file_path):
        file_name = file_path.split('/')[-1]
        if '.' in file_name:
            file_name = file_name.rsplit('.', 1)[0]
        standard_name = file_name.replace(' ', '_').lower()
        return standard_name

    def parse_gedcom(self):
        with open(self.file_name, 'r', encoding='utf-8-sig') as f:
            gedcom_lines = f.readlines()

        current_dataset = None
        npfx_count = 0
        total_count = 0
        for line in gedcom_lines:
            parts = line.strip().split(' ', 2)
            level = int(parts[0])
            tag = parts[1]
            value = parts[2] if len(parts) > 2 else None

            if level == 0 and tag.startswith('@') and tag.endswith('@') and value == 'INDI':
                total_count += 1
                current_dataset = GedcomDataset(tag)
                self.gedcom_datasets.append(current_dataset)
            elif current_dataset is not None:
                if level == 1 and tag in ['NAME', 'FAMC']:
                    current_key = tag
                    current_dataset.add_extractable_detail(current_key, value)
                elif level == 2 and tag == 'NPFX':
                    npfx_count += 1
                    current_dataset.add_extractable_detail(tag, value)

        print(f'Found {total_count} total records')
        print(f'Found {npfx_count} NPFX tags')

        for dataset in self.gedcom_datasets:
            if dataset.get_extractable_NPFX():
                self.filter_pool.append(dataset)

    def create_extractable_details_dict(self):
        extractable_details_dict = {}
        for i, dataset in enumerate(self.filter_pool):
            if i >= 500:
                break
            row = []
            row.append(dataset.get_gen_person())
            row.append(dataset.get_anchor_gen1())
            row.append(dataset.get_extractable_cm())
            row.append(dataset.get_extractable_sort())
            row.append(dataset.get_extractable_FAMC())
            extractable_details_dict[dataset.get_gen_person()] = row
        return extractable_details_dict

class GedcomDataset:
    def __init__(self, gen_person):
        self.gen_person = gen_person
        self.extractable_detail = {}

    def add_extractable_detail(self, key, value):
        self.extractable_detail[key] = value

    def get_gen_person(self):
        name = self.extractable_detail.get('NAME', '')
        first_name, last_name = name.split('/', 1)
        first_name = first_name.split(' ')[0]
        last_name = last_name.rstrip('/')
        self.anchor_gen1 = last_name.replace(" ", "") + first_name.replace(" ", "")
        return self.gen_person.strip('@')

    def get_anchor_gen1(self):
        return self.anchor_gen1

    def get_extractable_NPFX(self):
        return self.extractable_detail.get('NPFX', '')

    def get_extractable_cm(self):
        npfx_value = self.extractable_detail.get('NPFX', '')
        if '&' in npfx_value:
            cm_value = npfx_value.split('&')[0].strip()
        else:
            cm_value = npfx_value.strip()
        try:
            int(cm_value)
            return cm_value
        except ValueError:
            return 'error'

    def get_extractable_sort(self):
        # Extract and return the extractable_sort (right side of NPFX split at "&") from the gedcom_dataset
        npfx_value = self.extractable_detail.get('NPFX', '')
        if '&' in npfx_value:
            sort_value = npfx_value.split('&')[1].strip()
            return sort_value
        else:
            return ''

    def get_extractable_FAMC(self):
        return self.extractable_detail.get('FAMC', '').strip('@')

def select_gedcom_file():
    # Return the hardcoded path directly
    return '/content/yates-one-name-study.ged'

# Call the function to let the user select the GEDCOM file
gedcom_file_path = select_gedcom_file()
if gedcom_file_path:
    # Use the selected GEDCOM file path to create an instance of the Gedcom class
    gedcom_instance = Gedcom(gedcom_file_path)
    gedcom_instance.parse_gedcom()
    # Rest of the processing using the Gedcom instance
else:
    print("No GEDCOM file selected. Exiting.")

# Section 2
from gedcom.element.individual import IndividualElement
from gedcom.parser import Parser
import pandas as pd

# Function definitions
def extract_id(record):
    id_start = record.find('@') + 1
    id_end = record.find('@', id_start)
    return record[id_start:id_end]

def extract_name(record):
    name_start = record.find('1 NAME ') + 6
    name_end = record.find('\n', name_start)
    name = record[name_start:name_end]
    first_name, last_name = name.split('/', 1)
    last_name = last_name.rstrip('/')
    return last_name.replace(" ", "") + first_name.replace(" ", "")

def find_parents(individual_id, generation, records):
    if individual_id not in records:  # check if key exists
        return
    record = records[individual_id]  # constant-time lookup
    famc_start = record.find('1 FAMC @') + 8
    famc_end = record.find('@', famc_start)
    famc_id = record[famc_start:famc_end]
    if famc_id not in records:  # family not in records
        return

    fam_record = records[famc_id]  # constant-time lookup
    wife_start = fam_record.find('1 WIFE @') + 8
    wife_end = fam_record.find('@', wife_start)
    mother_id = fam_record[wife_start:wife_end]

    husb_start = fam_record.find('1 HUSB @') + 8
    husb_end = fam_record.find('@', husb_start)
    father_id = fam_record[husb_start:husb_end]

    if mother_id and mother_id in records:  # check if key exists and is not empty
        mother_record = records[mother_id]  # constant-time lookup
        mother_name = extract_name(mother_record)
    else:
        mother_name = None

    if father_id and father_id in records:  # check if key exists and is not empty
        father_record = records[father_id]  # constant-time lookup
        father_name = extract_name(father_record)
    else:
        father_name = None

    if mother_name is not None and father_name is not None:
        parent_pair = father_name + "&" + mother_name
        if parent_pair not in visited_pairs:
            visited_pairs.add(parent_pair)
            generation_table.loc[len(generation_table)] = [generation, parent_pair]

    if mother_id:
        find_parents(mother_id, generation + 1, records)

    if father_id:
        find_parents(father_id, generation + 1, records)

def process_individual(individual_id):
    global generation_table
    generation_table = pd.DataFrame(columns=['Generation', 'Parent Pair'])
    global visited_pairs
    visited_pairs = set()

    find_parents(individual_id, 1, records)

    generation_table = generation_table.sort_values('Generation', ascending=False).reset_index(drop=True)

    individual_data = {}
    dataset = next((dataset for dataset in gedcom.gedcom_datasets if dataset.get_gen_person() == individual_id), None)
    if dataset:
        individual_data['cM'] = dataset.get_extractable_cm()
        individual_data['Sort'] = dataset.get_extractable_sort()
    individual_data['Parent Pairs A10'] = '|'.join([f"{row['Parent Pair']}" for index, row in generation_table[:10].iterrows()])
    return individual_data




    individual_data['generations_source'] = '|'.join([f"{row['Parent Pair']}" for index, row in generation_table.iterrows()])
    individual_data['generations_A10'] = '|'.join([f"{row['Parent Pair']}" for index, row in generation_table.iterrows()])
    individual_data['generations_B10'] = '|'.join([f"{row['Parent Pair']}" for index, row in generation_table.iterrows()]) if len(generation_table) > 10 else ''
    return individual_data


# Run the process_individual function for each individual in the individuals list and create a combined DataFrame
combined_df_rows=[]
for name, individual_id in individuals:
    individual_data = process_individual(individual_id)
    cm = individual_data['cM']
    sort = individual_data['Sort']
    parent_pairs_a10 = individual_data['Parent Pairs A10']
    combined_df_rows.append([name, individual_id, cm, sort, parent_pairs_a10])

combined_df = pd.DataFrame(combined_df_rows, columns=['Last Name', 'ID#', 'cM', 'Sort', 'Parent Pairs A10'])

# Adjust index to start from 1 instead of 0
combined_df.index = combined_df.index + 1

# Print all records from the DataFrame
print(combined_df)

# Export the combined_df DataFrame to an Excel file
combined_df.to_excel('output.xlsx', index=False)

Found 50776 total records
Found 375 NPFX tags
Total records found: 375
Found 50776 total records
Found 375 NPFX tags
              Last Name     ID#  cM             Sort  \
1          YatesWilliam  I13817  11      yates,johnh   
2            HuntKelsey  I21743  15    yates,robertd   
3         JohnsonDonald  I23678  53    yates,andreal   
4              HudsonJL  I26925  27    yates,andreal   
5           ChurchDebra  I31861  19  yates,patricial   
..                  ...     ...  ..              ...   
371       BeaversLaNell  I50494  16     yates,ronald   
372       StiversKaylee  I50511  14      yatesronald   
373  FehrenbacherAngela  I50522  17      yatesronald   
374      PetersonHannah  I50535  14     yates,ronald   
375      NicholsTheresa  I50551   8     yates,ronald   

                                      Parent Pairs A10  
1    YatesJohnJ.&YatesAgnesAnn|YatesWilliamWesley&B...  
2    YatesRichard&AshendonJoan|TichborneJohn&Martin...  
3    YatesRichard&AshendonJoan|Tichborn

In [1]:
!pip install pandas
!pip install python-gedcom